# QandA Demo

## Overview
This notebook demonstrates the `QandA` class from `qanda.py`, a RAG system for extracting information from coroner's reports. It uses local LLMs (Ollama) for secure processing, aligning with project aims: automating extraction of road fatality details (e.g., deceased info, causes, circumstances) from unstructured PDFs.

Key demo steps: Initialize, query (basic/verbose), evaluate with BERTScore, and compare models.

### Prerequisites
- `conda activate coroner_env`
- Ollama running with `mxbai-embed-large`, `gemma3`, `llama3.2`, and `phi4-mini` pulled.
- `jsondata/Rodier-Finding.jsonl` available (pre-processed chunks from PDF).

## Imports

In [1]:
import os
import pandas as pd
from pathlib import Path

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.vectorstores import InMemoryVectorStore

from qanda import QandA
from scores import calculate_bertscore_df

## QandA Object

`QandA` encapsulates the RAG pipeline: loads JSONL chunks, embeds into vector store, retrieves top-K contexts, generates LLM answers grounded in documents. Supports project deliverables (2.1.II: RAG integration for contextual queries).

## Initialize QandA

Parameters:
- `file_path`: JSONL with chunks/metadata.
- `gen_model`: LLM (e.g., `gemma3`).
- `embed_model`: For semantic search.
- `vdb`: In-memory store for demo speed.
- `top_k`: Context chunks (3 balances relevance/token limits).
- `prompt`: Grounds answers in context (mitigates hallucinations, per Section 3).

In [2]:
FILE_PATH = Path("jsondata/Rodier-Finding.jsonl")
GEN_MODEL = "gemma3"
EMBED_MODEL = "mxbai-embed-large"
VDB = InMemoryVectorStore
TOP_K = 3

PROMPT = ChatPromptTemplate.from_template(
    """Context information is below.
    \n---------------------\n
    {context}
    \n---------------------\n
    Given the context information and not prior knowledge, answer the query.\n
    Query: {input}\n
    Answer:\n""",
)

In [3]:
qanda = QandA(gen_model=GEN_MODEL,
              embed_model=EMBED_MODEL, 
              vdb=VDB,
              file_path=FILE_PATH,
              top_k=TOP_K,
              prompt=PROMPT)

Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.


## Usage Help

See docstring for `ask(question, verbose=False)`: Returns answer (or (answer, sources) if verbose).

In [4]:
help(qanda)

Help on QandA in module qanda object:

class QandA(builtins.object)
 |  QandA(gen_model, embed_model, vdb, file_path, top_k, prompt)
 |
 |  A class for performing question-answering tasks using a language model and a vector database.
 |
 |  Attributes:
 |      gen_model (str): The name of the language model to be used for generating answers.
 |      embed_model (str): The name of the embedding model to be used for generating embeddings.
 |      vdb (str): The name of the vector database to be used for storing and retrieving documents.
 |      file_path (str): The path to the file containing the documents to be used for the question-answering task.
 |      top_k (int): The number of top-k most relevant documents to be retrieved for each question.
 |      prompt (str): The prompt to be used for the question-answering chain.
 |
 |  Methods:
 |      ask(question, verbose=False):
 |          Invokes the question-answering chain to generate an answer to the given question.
 |          Args:


## Querying Examples

Target key info: coroner, deceased, cause (per 2.1: structured outputs for analysis).

### Basic Queries

`ask(question)`: Direct answers for quick extraction.

In [5]:
QUESTIONS = ["Who is the coroner?", "Who is the deceased?", "What was the cause of death?"]
LLM_ANSWERS = []

for i, QUESTION in enumerate(QUESTIONS):
    ANSWER = qanda.ask(QUESTION)
    LLM_ANSWERS.append(ANSWER)
    print(f"Answer {i + 1}: ", ANSWER)

Answer 1:  Sarah Helen Linton, Deputy State Coroner.
Answer 2:  Frank Edward Rodier is the deceased.
Answer 3:  The cause of death remains unascertained. The report states, “his cause of death must remain unascertained.” It suggests he likely died from injuries sustained from the rocks, but this is not definitively established.


### Verbose Queries

`ask(question, verbose=True)`: Includes sources (text, page, doc) for auditing (2.1.II: source auditing).

In [6]:
for i, QUESTION in enumerate(QUESTIONS):
    ANSWER, SOURCES = qanda.ask(QUESTION, verbose=True)
    print(f"Question {i + 1}: {QUESTION}")
    print(f"Answer: {ANSWER}\n")
    print("Sources:")
    for src in SOURCES:
        print(f"  Source {src['source']}:")
        print(f"    Text: {src['text'][:200]}...")
        print(f"    Page: {src['page']}, Document: {src['document']}\n")
    print("-" * 50)

Question 1: Who is the coroner?
Answer: Sarah Helen Linton, Deputy State Coroner.

Sources:
  Source 1:
    Text: Counsel Appearing:
Senior Constable C Robertson assisted the Coroner .
Case(s) referred to in decision(s):
Nil...
    Page: 1, Document: data/Rodier-Finding.pdf

  Source 2:
    Text: [2024] WACOR 35
Coroners Act 1996 (Section 26(1))...
    Page: 2, Document: data/Rodier-Finding.pdf

  Source 3:
    Text: [2024] WACOR 35
JURISDICTION
CORONER'S COURT OF WESTERN AUSTRALIA
ACT
CORONERS ACT 1996
CORONER
SARAH HELEN LINTON, DEPUTY STATE CORONER
HEARD
14 AUGUST 2024
DELIVERED
14 AUGUST 2024
FILE NO/S
CORC 32...
    Page: 1, Document: data/Rodier-Finding.pdf

--------------------------------------------------
Question 2: Who is the deceased?
Answer: Frank Edward Rodier

Sources:
  Source 1:
    Text: IS DEATH ESTABLISHED?
17. As is clear from the above; I am satisfied beyond reasonable doubt that Frank Rodier is deceased and that he died on 25 1975 in the sea after he was washed o

## Evaluation

BERTScore on basic answers vs. ground-truth (from PDF manual review). Measures semantic accuracy for consistency (Benefits III).

In [7]:
CORRECT_ANSWERS = ["Sarah Helen Linton", "Frank Edward Rodier", "unascertained"]

data = {
    'FILENAME': ['Rodier-Finding'] * len(QUESTIONS),
    'MODEL': [GEN_MODEL] * len(QUESTIONS),
    'QUESTION': QUESTIONS,
    'CORRECT_ANSWER': CORRECT_ANSWERS,
    'LLM_ANSWER': LLM_ANSWERS
}

df = pd.DataFrame(data)
scores_df = calculate_bertscore_df(df)

display(scores_df)
print(f"Average F1 Score: {scores_df['BERT_F1'].mean():.3f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.85 seconds, 1.62 sentences/sec


,FILENAME,MODEL,QUESTION,CORRECT_ANSWER,LLM_ANSWER,BERT_PRECISION,BERT_RECALL,BERT_F1
0,Rodier-Finding,gemma3,Who is the coroner?,Sarah Helen Linton,"Sarah Helen Linton, Deputy State Coroner.",0.888055,0.959326,0.922316
1,Rodier-Finding,gemma3,Who is the deceased?,Frank Edward Rodier,Frank Edward Rodier is the deceased.,0.913599,0.961740,0.937052
2,Rodier-Finding,gemma3,What was the cause of death?,unascertained,The cause of death remains unascertained. The ...,0.814397,0.838729,0.826384


Average F1 Score: 0.895


# ------------------------------------------------------------------------------ 
# Method 4: Compare answers of different models
# ------------------------------------------------------------------------------

This section initializes QandA instances for multiple LLMs and compares their responses to a targeted question on the report. It evaluates semantic similarity via BERTScore, highlighting model strengths for project scalability (Benefit II: Associative studies across models).

In [8]:
LLAMA = "llama3.2"
GEMMA = "gemma3"
PHI   = "phi4-mini"

qanda_llama = QandA(gen_model=LLAMA,
                    embed_model=EMBED_MODEL, 
                    vdb=VDB,
                    file_path=FILE_PATH,
                    top_k=TOP_K,
                    prompt=PROMPT)

qanda_gemma = QandA(gen_model=GEMMA,
                    embed_model=EMBED_MODEL, 
                    vdb=VDB,
                    file_path=FILE_PATH,
                    top_k=TOP_K,
                    prompt=PROMPT)

qanda_phi = QandA(gen_model=PHI,
                  embed_model=EMBED_MODEL, 
                  vdb=VDB,
                  file_path=FILE_PATH,
                  top_k=TOP_K,
                  prompt=PROMPT)

QUESTION = "What activity was implicated in the cause of death?"
CORRECT_ANSWER = "Fishing"
LLM_ANSWERS = []

for i, qanda_model in enumerate([qanda_llama, qanda_gemma, qanda_phi]):
    ANSWER = qanda_model.ask(QUESTION)
    LLM_ANSWERS.append(ANSWER)
    print(f"Answer {i + 1}: ", ANSWER)

data = {
    'FILENAME': ['Rodier-Finding'] * 3,
    'MODEL': [LLAMA, GEMMA, PHI],
    'QUESTION': [QUESTION] * 3,
    'CORRECT_ANSWER': [CORRECT_ANSWER] * 3,
    'LLM_ANSWER': LLM_ANSWERS
}

df = pd.DataFrame(data)

scores_df = calculate_bertscore_df(df)

print(scores_df.columns)

print(scores_df)

Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.
Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.
Initializing, please wait...
Loading jsondata\Rodier-Finding.jsonl
Question Answer chain ready.
Answer 1:  Based on the provided context information, the activity implicated in the cause of death is fishing. Frank Rodier was washed off the rocks while fishing with friends, which led to his death in 1975.
Answer 2:  Fishing with friends.
Answer 3:  The activity implied to be related to Frank Rodier's cause of death is fishing with friends. He died after being washed off rocks while engaging in this activity on 25 July 1975. The manner suggested by the context for his demise appears accidental, as he was likely overwhelmed or unable to swim back safely due to a sudden wave at sea during an outing that included swimming and diving activities among other pursuits related to fishing with friends.

In s

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 2.22 seconds, 1.35 sentences/sec
Index(['FILENAME', 'MODEL', 'QUESTION', 'CORRECT_ANSWER', 'LLM_ANSWER',
       'BERT_PRECISION', 'BERT_RECALL', 'BERT_F1'],
      dtype='object')
         FILENAME      MODEL  \
0  Rodier-Finding   llama3.2   
1  Rodier-Finding     gemma3   
2  Rodier-Finding  phi4-mini   

                                            QUESTION CORRECT_ANSWER  \
0  What activity was implicated in the cause of d...        Fishing   
1  What activity was implicated in the cause of d...        Fishing   
2  What activity was implicated in the cause of d...        Fishing   

                                          LLM_ANSWER  BERT_PRECISION  \
0  Based on the provided context information, the...        0.805051   
1                              Fishing with friends.        0.866990   
2  The activity implied to be related to Frank Ro...        0.768712   

   BERT_RECALL   BERT_F1  
0     0.821265  0.813078  
1     0.888260  0.877496  
2     0.866824  0.814825  


## Next Steps for WACRSR

- Bulk process: Loop over JSONL files for associative studies (Benefit II).
- Visualize: Aggregate outputs (e.g., Pandas for trends; Week 7-8).
- New reports: OCR preprocess (Week 3-5).

Enables tackling road fatalities scalably (Aim).